In [ ]:
#Declare variables
import datetime
import pandas as pd
import numpy as np
#Constants - Storage Cost, Units in Millions

MILLION = 10**6
df = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\At_Gas.csv',index_col=0,parse_dates=True)
df_input = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\Contract Details.csv')

#To extract price at a specific date
X = df.values
#print(X)
#print(df)

#Injection and withdrawal rates
specific_date = '2021-01-31' # Replace with whatever input date you want to check

#Scaling the Rates as per volume

class Contract():
    def __init__(self,qty_bought,specific_date,transaction_type,storage_cost,injection_cost,max_volume,df):
        self.qty_bought = qty_bought
        self.specific_date = specific_date
        self.transaction_type = transaction_type
        self.storage_cost = storage_cost
        self.injection_cost = injection_cost
        self.max_volume = max_volume
        self.df = df

    def specific_rate(self):
        rate = []
        # Convert the specific date to a datetime object
        for i in range(0,len(self.specific_date)):
            valuetobeappenedintorate = self.df.loc[self.df.index == self.specific_date[i], 'Prices'].values
            if valuetobeappenedintorate.size > 0:
                
                rate.append(valuetobeappenedintorate[0] * MILLION)
            else: rate.append(0)

        return rate

    def price_calculation(self,current_rate):
        prices = []
        # print(self.qty_bought)
        # print(current_rate)
        # print(len(self.qty_bought))
        for i in range(0,len(self.qty_bought)):
            prices.append(self.qty_bought[i] * current_rate[i])
            # if self.transaction_type[i] == 'I':
            #     prices[i] = prices[i] * (-1)
        return prices
    
    def inflow_outflow(self,buying_cost):
        new_buying_cost = []
        for i in range(0, len(self.transaction_type)):
            if self.transaction_type[i] == 'I':
                new_buying_cost.append(buying_cost[i] * (-1))
            elif self.transaction_type[i] == 'W':
                new_buying_cost.append(buying_cost[i])
        return new_buying_cost
    
    def storage_running(self):
        #Max storage = 6
        #Current storage = 4 + 3 = 7(New Inj Qty)
        active_storage = 1
        storage_sign = self.sign_operator(self.qty_bought)
        running_storage = [sum(storage_sign[:x+1]) for x,v in enumerate(storage_sign)]

        if sum([i < 0 or i > self.max_volume for i in running_storage]) > 0:
            raise Exception("Qty sold Exceeeds storage limit")
        return running_storage

    def calc_storage_cost(self,storage_running):
        for i in range(0,len(storage_running)):
            storage_running[i] = abs((storage_running[i] * self.storage_cost))
        return storage_running
    
    def conv_to_mill(self,value):
        if isinstance(value,list):
            for i in range (0,len(value)):
                value[i] = value[i] * MILLION
            return value
        elif isinstance(value,(int,float)):
            return value*MILLION
    
    def inject_withdraw_cost(self):
        inj_cost = []
        for i in range(0,len(self.qty_bought)):
            inj_cost.append(abs((self.qty_bought[i] * self.injection_cost)))
        return inj_cost
    def sign_operator(self,anything_you_want_changed):

        return [v*(-1) if self.transaction_type[y] == 'W' else v for y,v in enumerate(anything_you_want_changed)]
    def generate_contract(self):
        running_tally = self.storage_running()
        #print(running_tally,"storage Running")
        cal_storage = self.calc_storage_cost(running_tally)
        #print(cal_storage,"total storage cost")
        current_rate = self.specific_rate()
        #print(current_rate,"current Rate")
        current_prices = self.price_calculation(current_rate)
        #print(current_prices,"Currentprices")
        buying_cost = self.inflow_outflow(current_prices)
        #print(buying_cost, "buying cost")
        inj_cost = self.inject_withdraw_cost()
        #print(inj_cost,"inj_cost")

        total_buying_cost = sum(buying_cost)
        total_storage_cost = sum(cal_storage)
        total_inj_withdra_cost = sum(inj_cost)

        return total_buying_cost + total_storage_cost + total_inj_withdra_cost

qty_bought = df_input['Qty'].values
dates = df_input['Dates'].values
transaction_type = df_input['Transaction type'].values
print(qty_bought,dates,transaction_type)


def main():
    
    contract = Contract(qty_bought,dates,transaction_type,10000,10000,10,df)
    return

main()


[4 3 2 3] ['31-01-2021' '30-11-2021' '30-06-2022' '31-01-2023'] ['I' 'W' 'I' 'W']
[np.int64(4), np.int64(-3), np.int64(2), np.int64(-3)] enum
[np.int64(4), np.int64(1), np.int64(3), np.int64(0)] storage Running
[np.int64(40000), np.int64(10000), np.int64(30000), np.int64(0)] total storage cost
[np.float64(10900000.0), np.float64(11200000.0), np.float64(10400000.0), np.float64(12100000.0)] current Rate
[np.float64(43600000.0), np.float64(33600000.0), np.float64(20800000.0), np.float64(36300000.0)] Currentprices
[np.float64(-43600000.0), np.float64(33600000.0), np.float64(-20800000.0), np.float64(36300000.0)] buying cost
[np.int64(40000), np.int64(30000), np.int64(20000), np.int64(30000)] inj_cost
5700000.0


C:\Users\Nikhil\AppData\Local\Temp\ipykernel_22048\980564240.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('D:\Power BI thing\Forage Data sets\JPMC Data analyst\At_Gas.csv',index_col=0,parse_dates=True)
